In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader

from gift_cnn import gift_cnn

In [ ]:
dataset = np.load("dataset.npz")
X, y = dataset["X"], dataset["y"]

print("Number of images:", X.shape[0])
print("Number of labels:", y.shape[0])
print("Unique labels:", np.unique(y))


In [ ]:
random_state = 42
X = X / 255.0 

In [ ]:
#vislualize

#chosen image
k=10

shape = (X[k].reshape(20,20))
flatten = shape.flatten()


plt.imshow(shape, vmin=0, vmax=255, cmap="gray")
plt.show()

In [ ]:
#MLP

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

mlp = MLPClassifier(max_iter=1000, solver='adam', random_state=random_state, early_stopping=True)

paramgrid_mlp = {
    'max_iter': [500, 1000, 2000, 3000],
    'hidden_layer_sizes': [(100,50), (200,100,50), (300,200,100), (400,300,200,100)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0001, 0.001, 0.01],
    'activation': ['relu', 'tanh'],
    'learning_rate_init': [0.001, 0.01, 0.05]
}

grid_mlp = GridSearchCV(mlp, paramgrid_mlp, cv=5, n_jobs=-1)
grid_mlp.fit(X_train, y_train)



y_pred_mlp = grid_mlp.predict(X_test)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

print("Best parameters for MLP:", grid_mlp.best_params_)
print("Best MLP Accuracy:", accuracy_mlp)

y_train_pred_mlp = grid_mlp.predict(X_train)
accuracy_train_mlp = accuracy_score(y_train, y_train_pred_mlp)
print("Training Accuracy:", accuracy_train_mlp)


print("Test Accuracy:", accuracy_mlp)

plt.plot(grid_mlp.best_estimator_.loss_curve_)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("MLP Training Loss Curve")
plt.show()

In [ ]:
X_tensor = torch.tensor(X, dtype=torch.float32).view(-1, 1, 20, 20)
y_tensor = torch.tensor(y, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=random_state
)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model = gift_cnn()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 20
loss_history = []

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train() 
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    loss_history.append(epoch_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

In [ ]:
def compute_accuracy(loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

train_acc = compute_accuracy(train_loader)
test_acc = compute_accuracy(test_loader)

print(f"Train Accuracy: {train_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")

In [ ]:
plt.plot(loss_history, marker='o')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("CNN Training Loss Curve")
plt.show()


In [ ]:
dataset = np.load("dataset_corrupted.npz")
X, y = dataset["X"], dataset["y"]

In [ ]:


# Load dataset
dataset = np.load("dataset.npz")
X, y = dataset["X"], dataset["y"]

# Normalize and reshape images
X = X / 255.0
X_tensor = torch.tensor(X, dtype=torch.float32).view(-1, 1, 20, 20)
y_tensor = torch.tensor(y, dtype=torch.long)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42
)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Instantiate model, loss function, optimizer
model = gift_cnn()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to compute accuracy
def compute_accuracy(loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Training loop
num_epochs = 20
loss_history = []
train_acc_history = []
test_acc_history = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    loss_history.append(epoch_loss)

    train_acc = compute_accuracy(train_loader)
    test_acc = compute_accuracy(test_loader)
    train_acc_history.append(train_acc)
    test_acc_history.append(test_acc)

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | "
          f"Train Acc: {train_acc*100:.2f}% | Test Acc: {test_acc*100:.2f}%")

# Plot training loss
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(loss_history, label='Training Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("CNN Training Loss")
plt.legend()

# Plot training and test accuracy
plt.subplot(1,2,2)
plt.plot(train_acc_history, label='Train Accuracy')
plt.plot(test_acc_history, label='Test Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("CNN Accuracy")
plt.legend()
plt.show()
